In [108]:
# In order to run this script, ensure you have installed the required dependencies in the requirements.txt file.
# For more information, refer to the README.md file.
import pandas as pd
import numpy as np
import os
from scipy.interpolate import interp1d

In [109]:
DATE_COL="Date - Date"

# load csvs
maximum_water_level = pd.read_csv("./data/raw/LakeOntario_HistoricalMax_WaterLevel.csv")
minimum_water_level = pd.read_csv("./data/raw/LakeOntario_HistoricalMin_WaterLevel.csv")

maximum_outflow = pd.read_csv("./data/raw/LakeOntario_HistoricalMax_WaterOutflow.csv")
minimum_outflow = pd.read_csv("./data/raw/LakeOntario_HistoricalMin_WaterOutflow.csv")

hydro_data_2024 = pd.read_csv("./data/processed/hydro_data_2024_hourly.csv", index_col=0, parse_dates=True)

# convert date columns to datetime
maximum_water_level[DATE_COL] = pd.to_datetime(maximum_water_level[DATE_COL])
minimum_water_level[DATE_COL] = pd.to_datetime(minimum_water_level[DATE_COL])

maximum_outflow[DATE_COL] = pd.to_datetime(maximum_outflow[DATE_COL])
minimum_outflow[DATE_COL] = pd.to_datetime(minimum_outflow[DATE_COL])


# set date columns as index
maximum_water_level.set_index(DATE_COL, inplace=True)
minimum_water_level.set_index(DATE_COL, inplace=True)
maximum_outflow.set_index(DATE_COL, inplace=True)
minimum_outflow.set_index(DATE_COL, inplace=True)

make it hourly

In [110]:
# reindex to hourly frequency and fill missing timestamps with the previous known value
start = pd.Timestamp("2024-01-01 00:00:00")
# end is set to the last timestamp in 2024 plus one minute since datasets are recorded hourly plus one minute
end = pd.Timestamp("2025-01-01 00:00:00")
hourly_index = pd.date_range(start=start, end=end, freq="h")

maximum_water_level = maximum_water_level.reindex(hourly_index)
maximum_water_level = maximum_water_level.ffill()

minimum_water_level = minimum_water_level.reindex(hourly_index)
minimum_water_level = minimum_water_level.ffill()

maximum_outflow = maximum_outflow.reindex(hourly_index)
maximum_outflow = maximum_outflow.ffill()

minimum_outflow = minimum_outflow.reindex(hourly_index)
minimum_outflow = minimum_outflow.ffill()


In [111]:

df_new = pd.DataFrame(index=hourly_index)

MAX_WATER_LEVEL_COL = "Maximum Water Level (meters)"
MIN_WATER_LEVEL_COL = "Minimum Water Level (meters)"
MAX_OUTFLOW_COL = "Maximum Outflow (m^3/s)"
MIN_OUTFLOW_COL = "Minimum Outflow (m^3/s)"
UPSTREAM_LEVEL_COL = "Upstream Water Level (meters)"
DOWNSTREAM_LEVEL_COL = "Downstream Water Level (meters)"

df_new[UPSTREAM_LEVEL_COL] = hydro_data_2024["upstream"]
df_new[DOWNSTREAM_LEVEL_COL] = hydro_data_2024["downstream"]
df_new["x"] = hydro_data_2024["x"]
df_new["h"] = hydro_data_2024["h"]
df_new[MAX_WATER_LEVEL_COL] = maximum_water_level["m"]
df_new[MIN_WATER_LEVEL_COL] = minimum_water_level["m"]
df_new[MAX_OUTFLOW_COL] = maximum_outflow["m^3/s"]
df_new[MIN_OUTFLOW_COL] = minimum_outflow["m^3/s"]
    
df_new.to_csv("./data/processed/hydro_data_2024_with_historical_levels.csv")

df_new

,Upstream Water Level (meters),Downstream Water Level (meters),x,h,Maximum Water Level (meters),Minimum Water Level (meters),Maximum Outflow (m^3/s),Minimum Outflow (m^3/s)
2024-01-01 00:00:00,72.880000,46.810000,1,25,75.19,73.74,10470.0,4590.0
2024-01-01 01:00:00,72.880000,46.812000,1,25,75.19,73.74,10470.0,4590.0
2024-01-01 02:00:00,72.870000,46.811000,1,25,75.19,73.74,10470.0,4590.0
2024-01-01 03:00:00,72.870000,46.814000,1,25,75.19,73.74,10470.0,4590.0
2024-01-01 04:00:00,72.870000,46.816000,1,25,75.19,73.74,10470.0,4590.0
...,...,...,...,...,...,...,...,...
2024-12-31 20:00:00,73.050000,46.894000,1,25,75.18,73.74,9540.0,4790.0
2024-12-31 21:00:00,73.060000,46.918000,1,25,75.18,73.74,9540.0,4790.0
2024-12-31 22:00:00,73.080000,46.927000,1,25,75.18,73.74,9540.0,4790.0
2024-12-31 23:00:00,73.080000,46.910000,1,25,75.18,73.74,9540.0,4790.0


In [112]:
upstream = lambda x: 0.0001*x + 72.301

upstream_inverse = lambda y: (y - 72.301) / 0.0001

max_outflow_from_upstream = upstream_inverse(df_new[MAX_WATER_LEVEL_COL])
constrainted_max_outflow = pd.concat([df_new[MIN_OUTFLOW_COL], max_outflow_from_upstream], axis=1).max(axis=1)
constrainted_max_outflow = pd.concat([df_new[MAX_OUTFLOW_COL], constrainted_max_outflow], axis=1).min(axis=1)



We demonstrate here that all of the outflows from the upstream inverse correlation always yield an outflow higher than the maximum historical dataset. Leading to our new optimized outflow to be the maximum historical dataset itself.

In [113]:
# demonstrate that all values from inverse correlation are higher than historical maximum outflow
print((max_outflow_from_upstream > df_new[MAX_OUTFLOW_COL]).all())

True


In [114]:
df_new["Optimized Outflow (m^3/s)"] = constrainted_max_outflow
df_new

,Upstream Water Level (meters),Downstream Water Level (meters),x,h,Maximum Water Level (meters),Minimum Water Level (meters),Maximum Outflow (m^3/s),Minimum Outflow (m^3/s),Optimized Outflow (m^3/s)
2024-01-01 00:00:00,72.880000,46.810000,1,25,75.19,73.74,10470.0,4590.0,10470.0
2024-01-01 01:00:00,72.880000,46.812000,1,25,75.19,73.74,10470.0,4590.0,10470.0
2024-01-01 02:00:00,72.870000,46.811000,1,25,75.19,73.74,10470.0,4590.0,10470.0
2024-01-01 03:00:00,72.870000,46.814000,1,25,75.19,73.74,10470.0,4590.0,10470.0
2024-01-01 04:00:00,72.870000,46.816000,1,25,75.19,73.74,10470.0,4590.0,10470.0
...,...,...,...,...,...,...,...,...,...
2024-12-31 20:00:00,73.050000,46.894000,1,25,75.18,73.74,9540.0,4790.0,9540.0
2024-12-31 21:00:00,73.060000,46.918000,1,25,75.18,73.74,9540.0,4790.0,9540.0
2024-12-31 22:00:00,73.080000,46.927000,1,25,75.18,73.74,9540.0,4790.0,9540.0
2024-12-31 23:00:00,73.080000,46.910000,1,25,75.18,73.74,9540.0,4790.0,9540.0


In [115]:
x = df_new["x"]
Qdot = df_new["Optimized Outflow (m^3/s)"]
h = df_new["h"]
ro = 998 # kg/m^3
g = 9.81 # m/s^2
df_new["P"] = x * ro * Qdot * g * h
df_new["E"] = df_new["P"] * 3600  # Joules in one hour --> 3600 seconds = 1 hour
df_new.loc[df_new.index[0], "E"] = 0 # Set the first value to 0

Create a new column that computes the revenue of each period

In [116]:
df_new["E (kWh)"] = df_new["E"] / 3600000  # kWh

winter_months = [11, 12, 1, 2, 3, 4] # November to April
summer_months = [5, 6, 7, 8, 9, 10] # May to October

off_peak_rate = 0.098  # $/kWh
mid_peak_rate = 0.157  # $/kWh
on_peak_rate = 0.203  # $/kWh

off_peak_hours = list(range(19, 24)) + list(range(0, 7))  # 7 PM to 7 AM

winter_mid_peak_hours = list(range(11, 17)) # 11 AM to 5 PM
summer_mid_peak_hours = list(range(7, 11)) + list(range(17, 19)) # 7 AM to 11 AM and 5 PM to 7 PM


def TOU_rate(timestamp):
    month = timestamp.month
    hour = timestamp.hour
    weekday = timestamp.weekday()  # Monday=0, Sunday=6

    # Weekend or off-peak hours
    if weekday >= 5 or hour in off_peak_hours:  # Weekend
        return off_peak_rate  # Off-peak rate
    
    # mid peak hours
    if hour in winter_mid_peak_hours and month in winter_months:
        return mid_peak_rate
    if hour in summer_mid_peak_hours and month in summer_months:
        return mid_peak_rate

    # on peak hours if not in any of the above
    return on_peak_rate
df_new["Rate ($/kWh)"] = df_new.index.map(TOU_rate)
df_new["Revenue ($)"] = df_new["E (kWh)"] * df_new["Rate ($/kWh)"]

In [117]:
df_new["Revenue ($)"]
df_new["Revenue ($)"].sum()

np.float64(2669312365.83657)

In [118]:
df_new.to_csv("./data/processed/hydro_data_2024_with_optimized_outflow_and_revenue.csv")
df_new

,Upstream Water Level (meters),Downstream Water Level (meters),x,h,Maximum Water Level (meters),Minimum Water Level (meters),Maximum Outflow (m^3/s),Minimum Outflow (m^3/s),Optimized Outflow (m^3/s),P,E,E (kWh),Rate ($/kWh),Revenue ($)
2024-01-01 00:00:00,72.880000,46.810000,1,25,75.19,73.74,10470.0,4590.0,10470.0,2.562632e+09,0.000000e+00,0.000,0.098,0.00000
2024-01-01 01:00:00,72.880000,46.812000,1,25,75.19,73.74,10470.0,4590.0,10470.0,2.562632e+09,9.225475e+12,2562631.965,0.098,251137.93257
2024-01-01 02:00:00,72.870000,46.811000,1,25,75.19,73.74,10470.0,4590.0,10470.0,2.562632e+09,9.225475e+12,2562631.965,0.098,251137.93257
2024-01-01 03:00:00,72.870000,46.814000,1,25,75.19,73.74,10470.0,4590.0,10470.0,2.562632e+09,9.225475e+12,2562631.965,0.098,251137.93257
2024-01-01 04:00:00,72.870000,46.816000,1,25,75.19,73.74,10470.0,4590.0,10470.0,2.562632e+09,9.225475e+12,2562631.965,0.098,251137.93257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 20:00:00,73.050000,46.894000,1,25,75.18,73.74,9540.0,4790.0,9540.0,2.335006e+09,8.406020e+12,2335005.630,0.098,228830.55174
2024-12-31 21:00:00,73.060000,46.918000,1,25,75.18,73.74,9540.0,4790.0,9540.0,2.335006e+09,8.406020e+12,2335005.630,0.098,228830.55174
2024-12-31 22:00:00,73.080000,46.927000,1,25,75.18,73.74,9540.0,4790.0,9540.0,2.335006e+09,8.406020e+12,2335005.630,0.098,228830.55174
2024-12-31 23:00:00,73.080000,46.910000,1,25,75.18,73.74,9540.0,4790.0,9540.0,2.335006e+09,8.406020e+12,2335005.630,0.098,228830.55174
